In [ ]:
import os
import numpy as np
import trimesh
import pyrender
from PIL import Image

# ── CONFIGURATION ────────────────────────────────────────────────────────
INPUT_DIR       = "/home/jeans/win/aaaJAIST/resources/LOD_data_50"
OUTPUT_ROOT     = "../resources/LOD_images"

LOD_LEVELS      = [1, 2, 3]        # only lod1, lod2, lod3
AZIMUTH_STEP    = 15              # degrees per shot
ELEVATIONS      = [0, 15, 30]     # elevation angles
IMAGE_SIZE      = (800, 600)      # width, height
SHOW_Y_AXIS     = True            # giant Y-axis box
DIST_FACTOR     = 2.0             # camera distance = radius * this

# ICP settings (same as your original)
ICP_SAMPLES     = 3000
ICP_SCALE       = False
ICP_FIRST       = 1
ICP_FINAL       = 30
CENTER_METHOD   = "bbox"          # only bbox for now
# ─────────────────────────────────────────────────────────────────────────

def load_scene(scene_id: int, lod: int) -> trimesh.Scene:
    """Load a multi-mesh OBJ as a trimesh.Scene (preserving textures)."""
    path = os.path.join(INPUT_DIR, str(scene_id), f"lod{lod}.obj")
    return trimesh.load(path)  # returns a Scene when OBJ has multiple parts

def extract_mesh(scene: trimesh.Scene) -> trimesh.Trimesh:
    """Concatenate all geometries for ICP fitting only."""
    return trimesh.util.concatenate(list(scene.geometry.values()))

def get_register_matrix(mesh_to_move, other):
    """
    Exactly your original ICP call.
    Returns (4×4 matrix, cost).
    """
    mat, cost = trimesh.registration.mesh_other(
        mesh_to_move,
        other,
        samples=ICP_SAMPLES,
        scale=ICP_SCALE,
        icp_first=ICP_FIRST,
        icp_final=ICP_FINAL
    )
    return mat, cost

def register_and_center(scenes: dict):
    """
    scenes: dict[lod] = trimesh.Scene
    1) extract meshes for lod1/2/3
    2) compute lod2→lod1, lod3→lod2
    3) apply transforms to scenes[2] and scenes[3]
    4) compute combined lod3→lod1 and apply
    5) center all scenes by bbox center of lod1
    """
    # extract
    m1 = extract_mesh(scenes[1])
    m2 = extract_mesh(scenes[2])
    m3 = extract_mesh(scenes[3])

    # ICP
    M21, _ = get_register_matrix(m2, m1)
    M32, _ = get_register_matrix(m3, m2)
    M31 = M21.dot(M32)

    # apply to scenes
    scenes[2].apply_transform(M21)
    scenes[3].apply_transform(M31)

    # center by bbox center of lod1
    if CENTER_METHOD == "bbox":
        mn, mx = scenes[1].bounds
        center = (mn + mx) * 0.5
    else:  # centroid if ever needed
        center = scenes[1].centroid

    for s in scenes.values():
        s.apply_translation(-center)

def look_at_matrix(eye, target, up):
    """Your original LookAt → camera-to-world matrix."""
    f = (target - eye); f /= np.linalg.norm(f)
    if np.isclose(np.linalg.norm(np.cross(f, up)), 0):
        up = np.array([0,0,1]) if np.isclose(abs(np.dot(f,[0,1,0])),1) else np.array([0,1,0])
    s = np.cross(f, up); s /= np.linalg.norm(s)
    u = np.cross(s, f);   u /= np.linalg.norm(u)
    view = np.array([
        [ s[0],  s[1],  s[2], -np.dot(s, eye)],
        [ u[0],  u[1],  u[2], -np.dot(u, eye)],
        [-f[0], -f[1], -f[2],  np.dot(f, eye)],
        [   0 ,    0 ,    0 ,              1 ]
    ])
    return np.linalg.inv(view)

def add_lights(scene: pyrender.Scene):
    """Same 3-point directional lights as before."""
    intensity = 3.0
    key = pyrender.DirectionalLight(np.ones(3), intensity)
    scene.add(key, pose=np.array([[0,0,1,2],[0,1,0,2],[1,0,0,2],[0,0,0,1]]))
    fill = pyrender.DirectionalLight(np.ones(3), intensity*0.5)
    scene.add(fill,pose=np.array([[0,0,-1,-2],[0,1,0,1],[-1,0,0,-2],[0,0,0,1]]))
    back = pyrender.DirectionalLight(np.ones(3), intensity*0.3)
    scene.add(back,pose=np.array([[1,0,0,-2],[0,0,1,-2],[0,1,0,2],[0,0,0,1]]))

def capture_orbit(scene_obj: trimesh.Scene, scene_id: int, lod: int):
    """
    Render from multiple azimuth/elevation angles, saving to OUTPUT_ROOT.
    Uses pyrender.Scene.from_trimesh_scene to keep textures.
    """
    # convert to pyrender
    pr_scene = pyrender.Scene.from_trimesh_scene(scene_obj)
    if SHOW_Y_AXIS:
        bbox = trimesh.primitives.Box(extents=[1,100000,1])
        wire = pyrender.Material(wireframe=True)
        pr_scene.add(pyrender.Mesh.from_trimesh(bbox, material=wire))
    add_lights(pr_scene)

    r = pyrender.OffscreenRenderer(*IMAGE_SIZE)
    radius = extract_mesh(scene_obj).extents.max() / 2  # use mesh extents for distance
    target = np.zeros(3)

    for az in range(0, 360, AZIMUTH_STEP):
        for el in ELEVATIONS:
            a = np.deg2rad(az); e = np.deg2rad(el)
            x = radius * DIST_FACTOR * np.cos(e) * np.sin(a)
            y = radius * DIST_FACTOR * np.sin(e)
            z = radius * DIST_FACTOR * np.cos(e) * np.cos(a)
            eye = np.array([x,y,z])

            cam = pyrender.PerspectiveCamera(yfov=np.pi/3.0)
            pose = look_at_matrix(eye, target, np.array([0,1,0]))
            node = pr_scene.add(cam, pose=pose)

            color, _ = r.render(pr_scene)
            img = Image.fromarray(color[:,:,:3])

            out_dir = os.path.join(
                OUTPUT_ROOT,
                str(scene_id),
                f"lod{lod}",
                f"angle_{az}",
                f"elevation_{el}"
            )
            os.makedirs(out_dir, exist_ok=True)
            name = f"scene{scene_id}_lod{lod}_angle{az}_elevation{el}.png"
            img.save(os.path.join(out_dir, name))

            pr_scene.remove_node(node)

    r.delete()

def process_scene(scene_id: int):
    # 1) load scenes
    scenes = {lod: load_scene(scene_id, lod) for lod in LOD_LEVELS}
    # 2) register + center
    register_and_center(scenes)
    # 3) orbit-capture each LOD
    for lod, sc in scenes.items():
        capture_orbit(sc, scene_id, lod)

if __name__ == "__main__":
    for sid in range(0, 51):
        print(f"Processing scene {sid}…")
        process_scene(sid)
